# **Importando Bibliotecas**

In [4]:
import pandas as pd
from io import StringIO

In [6]:
with open("Dados/1410035501-noSymbol.csv", "r") as file:
    conteudo = file.read()

    texto_csv = ""

    conteudo_linhas = conteudo.split("\n")
    conteudo_linhas = conteudo_linhas[11:32]
    del conteudo_linhas[1]

    colunas = conteudo_linhas[0]

    lista_meses = colunas.split(",")
    meses = len(lista_meses[1:])

    for i, linha in enumerate(conteudo_linhas):

        valores = linha.split('"')

        valores = valores[1::2]

        valor_final = ";".join(valores)

        texto_csv += f"{valor_final}\n"


In [7]:
arquivo_csv = StringIO(texto_csv)

df = pd.read_csv(arquivo_csv, sep=";")

display(df)

,North American Industry Classification System (NAICS) 6,January 2022,February 2022,March 2022,April 2022,May 2022,June 2022,July 2022,August 2022,September 2022,...,November 2023,December 2023,January 2024,February 2024,March 2024,April 2024,May 2024,June 2024,July 2024,August 2024
0,"Total employed, all industries 7","19,280.1","19,615.3","19,639.8","19,692.9","19,736.5","19,705.0","19,740.1","19,728.6","19,727.8",...,"20,318.1","20,324.9","20,362.2","20,402.9","20,400.7","20,491.1","20,517.8","20,516.4","20,513.6","20,535.7"
1,Goods-producing sector 8,"3,999.1","4,040.6","4,061.4","4,069.7","4,054.7","4,068.7","4,081.8","4,100.2","4,080.8",...,"4,160.7","4,127.8","4,104.8","4,098.5","4,128.4","4,118.0","4,097.3","4,109.9","4,122.0","4,124.7"
2,Agriculture 9,251.4,252.3,252.0,252.7,254.5,257.5,254.7,264.1,261.1,...,253.6,240.8,235.0,229.0,226.5,221.1,224.3,236.6,239.0,240.5
3,"Forestry, fishing, mining, quarrying, oil and ...",328.1,328.8,337.1,334.3,334.4,326.3,328.1,330.0,326.0,...,330.0,332.2,330.7,334.6,338.1,345.8,348.8,351.8,356.8,350.3
4,Utilities,143.9,142.7,140.5,147.5,148.3,145.8,147.9,151.5,155.4,...,154.2,158.8,155.8,155.1,159.4,154.4,149.0,146.0,152.2,145.4
5,Construction,"1,491.5","1,526.6","1,539.7","1,543.1","1,547.4","1,560.2","1,566.5","1,562.3","1,559.9",...,"1,596.8","1,581.7","1,575.1","1,585.6","1,600.9","1,589.8","1,560.2","1,556.4","1,563.4","1,566.7"
6,Manufacturing,"1,784.2","1,790.3","1,792.2","1,792.2","1,770.1","1,779.0","1,784.6","1,792.3","1,778.4",...,"1,826.1","1,814.3","1,808.1","1,794.2","1,803.5","1,806.9","1,815.0","1,819.2","1,810.6","1,821.8"
7,Services-producing sector 12,"15,281.0","15,574.7","15,578.4","15,623.1","15,681.9","15,636.2","15,658.2","15,628.4","15,647.0",...,"16,157.5","16,197.1","16,257.5","16,304.4","16,272.4","16,373.1","16,420.5","16,406.4","16,391.6","16,411.1"
8,Wholesale and retail trade,"2,993.4","3,017.1","3,031.0","3,033.1","3,038.6","2,990.6","2,989.9","2,981.6","2,966.6",...,"2,948.0","2,932.1","2,963.4","2,946.6","2,923.5","2,929.6","2,923.3","2,913.0","2,868.9","2,882.8"
9,Transportation and warehousing,"1,007.8","1,012.2",981.8,977.4,972.2,976.8,997.5,983.1,969.2,...,"1,062.4","1,053.5","1,073.4","1,082.8","1,081.6","1,088.3","1,067.7","1,056.0","1,070.5","1,072.1"
